SEGMENTING AND CLUSTERING NEIGHBORHOODS IN TORONTO CITY

Introduction

PART 1: In this notebook, the first part of course 9 assignment for Peer review, the focus is on building the code to scrape the 
wikipedia page with the following link: 'https://en.wikipedia.org/wiki/List-of-postal-codes-of-Canada:M' . Find PART 2 at The Bottom.

In [ ]:
#Needed libraries
from bs4 import BeautifulSoup # A popular webscraping libraries among many others
import numpy as np
import pandas as pd # as library for data analysis

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
from geopy.geocoders import Nominatim #Converts an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize #Transform JSON file into a pandas dataframe

#matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#import k-means for clustering stage
from sklearn.cluster import KMeans

import folium # Map rendering library
from IPython.display import HTML,display

print('Libraries imported')

In [ ]:
url='https://en.wikipedia.org/wiki/List of postal codes of Canada:M'
lopcoc=requests.get(url)

In [ ]:
#create a soup object to retrieve html elements
soup=BeautifulSoup(lopcoc.text)

In [ ]:
all_links=soup.find_all('a')
for link in all_links:
    print(link.get('href'))

In [ ]:
#Manual inspection of all anchor tags(a) points to this link as possible external source
url="https://en.wikipedia.org/w/index.php?search=List+of+postal+codes+of+Canada%3AM&title=Special%3ASearch&fulltext=1"
resp=requests.get(url)

In [ ]:
#create a soup object to retrieve html elments after parsing by BeautifulSoup
soup=BeautifulSoup(resp.text)

In [ ]:
#provides a nice nested structure of html elements
print(soup.prettify())

In [ ]:
#further inspection of the parsed html page points to this possible container holding the right link
container_of_interest=soup.find(class_="searchresults mw-searchresults-has-iw")

In [ ]:
#fetch all 'a' tags for manual inspection to finda clue to possible link to the data table
soup.find_all('a')

In [ ]:
#Inspection of all links returned gives an indication of the right link with position index for all post codes
#in Canada starting from pos=0 for postal codes beginning with M (Toronto), and index in the list is 10
soup.find_all('a')[10]

In [ ]:
#This link was used to start a new web scraping as in the above
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
resp=requests.get(url)

In [ ]:
soup=BeautifulSoup(resp.text)

In [ ]:
print(soup.prettify())

In [ ]:
#Fortunately, there was only one table returned from this code
table=soup.findAll('table')

Getting table headers cell information in HTML element(denoted by the tags), which is found in table row element(tr) 
of a table header element(th) of a table element(tb). 
These elements can be fetched using BeautifulSoup object soup

In [ ]:
soup.findAll('tr', limit=1)

In [ ]:
#we skip the header colomns rows by setting the limit value=1
column_headers=[th.getText() for th in soup.findAll('tr', limit=1)]

In [ ]:
#is found in 30 rows starting from index location 0
data_rows=soup.findAll('tr')[0:30]

In [ ]:
#to retrieve data from all the rows we need to iterate over td in the rows
lopcot_data=[[td.getText() for td in data_rows[i].findAll('td')]
            for i in range(len(data_rows))]

In [ ]:
#after retrieving data we create a data passing data rows and data stored in the variable lopcot_data as arguments
#and include column names
df=pd.DataFrame(lopcot_data, columns=['PostalCode', 'Borough', 'Neighborhood'])

In [ ]:
# We clean the row data with special characters such as the new line('\n') seen at the end of the Neighborhood
#column values
df['Neighborhood']=df['Neighborhood'].str.replace('\n', '')

In [ ]:
df.head(30)

In [ ]:
#it is handy to manually create a new dataframe after conmining posstal codes. and neighborhoods as required by the assignment
df1=pd.DataFrame({'PostalCode':['M3A', 'M4A', 'M5A', 'M6A', 'M7A', 'M9A',
                               'M1B', 'M3B', 'M4B', 'M5B', 'M6B', 'M9B', 
                               'M1C'],
                 'Borough':['North York', 'North York', 'DownTown Toronto', 
                            'North York', "Queen's Park", 'Etobicoke', 'Scarborough',
                           'North York', 'East York', 'DownTown Toronto', 'North York',
                           'Etobicoke', 'Scarborough'],
                 'Neighborhood':['Parkhoods', 'Victoria Village', 'Harbourfront,Regent Park',
                                 'Lawrence Height,Lawrence Manor', "Queen's Park", 'Islington Avenue',
                                'Rouge,Malvern', 'Don Hills North', 'Woodbine Gardens,Parkview Hill',
                                'Ryerson,Garden District', 'Glencairn', 
                                 'Cloverdale,Islington,Martin Grove,Princess Gardens,West Dean Park',
                                'Highland Creek,Rouge Hill']}, columns=['PostalCode', 'Borough', 'Neighborhood'])
df1

In [ ]:
df1.shape

PART 2: Course 9 Assignment for Peer review. Now that a dataframe of the postal codes of each neighborhood has been built in in Part 1 above, along with the borough and neighborhood names, this part of the assignment focuses on using the Foursquare location data, by first getting the latitude and longitude coordinates of each neighborhood. The main objective here is to cluster the neighborhoods in Toronto, using the dataframe df1 derived above.

In [ ]:
#Download geographical coordinates of the neighborhoods
lat_lng_coords=pd.read_csv('https://cocl.us/Geospatial_data')
lat_lng_coords

In [59]:
#it is handy to manually create a new dataframe after conmining posstal codes. and neighborhoods as required by the assignment
neighborhoods_data=pd.DataFrame({'PostalCode':['M3A', 'M4A', 'M5A', 'M6A', 'M7A', 'M9A',
                               'M1B', 'M3B', 'M4B', 'M5B', 'M6B', 'M9B', 
                               'M1C'],
                 'Borough':['North York', 'North York', 'DownTown Toronto', 
                            'North York', "Queen's Park", 'Etobicoke', 'Scarborough',
                           'North York', 'East York', 'DownTown Toronto', 'North York',
                           'Etobicoke', 'Scarborough'],
                 'Neighborhood':['Parkhoods', 'Victoria Village', 'Harbourfront,Regent Park',
                                 'Lawrence Height,Lawrence Manor', "Queen's Park", 'Islington Avenue',
                                'Rouge,Malvern', 'Don Hills North', 'Woodbine Gardens,Parkview Hill',
                                'Ryerson,Garden District', 'Glencairn', 
                                 'Cloverdale,Islington,Martin Grove,Princess Gardens,West Dean Park',
                                'Highland Creek,Rouge Hill'],
                 'Latitude':[43.753259, 43.725882, 43.654260, 43.718518, 43.662301, 43.667856, 43.806686,
                            43.745906, 43.706397, 43.657162, 43.709577, 43.650943, 43.784535],
                 'Longitude':[-79.329656, -79.315572, -79.360639, -79.464763, -79.389494, -79.532242, -79.194353,
                             -79.352188, -79.309937, -79.378937, -79.445073, -79.554724, -79.160497]}, columns=['PostalCode', 'Borough',
                                                                                                               'Neighborhood', 'Latitude',
                                                                                                               'Longitude'])

neighborhoods_data

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkhoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,DownTown Toronto,"Harbourfront,Regent Park",43.654260,-79.360639
3,M6A,North York,"Lawrence Height,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
7,M3B,North York,Don Hills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
9,M5B,DownTown Toronto,"Ryerson,Garden District",43.657162,-79.378937


Use geopy library to get the latitude and longitude values of Toronto City. In order to define an instance of the geocoder, it's required to define a user_agent, here named as toronto_explorer.

In [38]:
#provide address name
address='Toronto City, Toronto'

geolocator=Nominatim(user_agent="toronto_explorer")
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude

print('The geographical coordinates of Toronto are {},{}.'.format(latitude, longitude))

The geographical coordinates of Toronto are 43.6524203,-79.3834045.


At this point we can start to utilize the Foursquare API to explore the neighborhoods and segment them

In [ ]:
#Define Foursquare credentials and version
CLIENT_ID='' #your foursquare ID
CLIENT_SECRET='' #your foursquare secret
VERSION='20190314' #your foursquare API version

print('Your credentials:')
print('CLIENT_ID:' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Time To Write The Code To Run The Above Function On Each Neighborhood And Create A New Dataframe Called toronto_venues

In [40]:
#Create a function to explore all neighborhoods in Toronto
def getNearbyVenues(names, latitudes, longitudes, radius=10000, LIMIT=500):
    venues_list=[]
    for name, lat, lng, in zip(names, latitudes, longitudes):
        print(name)
        
        #Create the API request URL
        url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID,
                                                                                                                                 CLIENT_SECRET,
                                                                                                                                 VERSION,
                                                                                                                                 lat,
                                                                                                                                 lng,
                                                                                                                                 radius,
                                                                                                                                 LIMIT)
        #Make the GET request
        results=requests.get(url).json()["response"]["groups"][0]['items']
        
        #Returns only relevant information for each nearby venue
        venues_list.append([(name,
                           lat,
                           lng,
                           v['venue']['name'],  
                           v['venue']['location']['lat'],
                           v['venue']['location']['lng'],
                           v['venue']['categories'][0]['name']) for v in results])
        
        nearby_venues=pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns=['Neighborhood',
                              'Neighborhood Latitude',
                              'Neighborhood Longitude',
                              'Venue',
                              'Venue Latitude',
                              'Venue Longitude',
                              'Venue Category']
        return(nearby_venues)

In [41]:
#the code
toronto_venues=getNearbyVenues(names=neighborhoods_data['Neighborhood'],
                              latitudes=neighborhoods_data['Latitude'],
                              longitudes=neighborhoods_data['Longitude']
                              )

Parkhoods


In [42]:
#Check the size of the resulting dataframe
print(northyork_venues.shape)
toronto_venues.head()

(100, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkhoods,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkhoods,43.753259,-79.329656,Darband Restaurant,43.755194,-79.348498,Middle Eastern Restaurant
2,Parkhoods,43.753259,-79.329656,LA Fitness,43.747665,-79.347077,Gym / Fitness Center
3,Parkhoods,43.753259,-79.329656,CF Shops at Don Mills,43.735011,-79.344787,Shopping Mall
4,Parkhoods,43.753259,-79.329656,Shawarma Empire,43.743405,-79.303649,Middle Eastern Restaurant


In [43]:
#Check how many venues were returned for each neighborhood
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Parkhoods,100,100,100,100,100,100


In [44]:
#Find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 58 uniques categories.


Time To Analyze Each Neighborhood

In [45]:
#onehot encoding
toronto_onehot=pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

#Add Neighborhood column back to dataframe
toronto_onehot['Neighborhood']=toronto_venues['Neighborhood']

#Move Neighborhood column to the first column
fixed_columns=[toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot=toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Afghan Restaurant,American Restaurant,Arts & Crafts Store,BBQ Joint,Bagel Shop,Bakery,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Café,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,Creperie,Dessert Shop,Dog Run,Farmers Market,Fish & Chips Shop,Fish Market,Flower Shop,French Restaurant,Furniture / Home Store,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Historic Site,History Museum,Hotel,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Liquor Store,Mediterranean Restaurant,Middle Eastern Restaurant,Movie Theater,Other Great Outdoors,Park,Pastry Shop,Pet Store,Restaurant,Rock Climbing Spot,Sandwich Place,Science Museum,Shopping Mall,Shopping Plaza,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Turkish Restaurant,Vietnamese Restaurant
0,Parkhoods,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Parkhoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Parkhoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Parkhoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,Parkhoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [46]:
#Examine the new dataframe
toronto_onehot.shape

(100, 59)

In [47]:
#Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
toronto_grouped=toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Afghan Restaurant,American Restaurant,Arts & Crafts Store,BBQ Joint,Bagel Shop,Bakery,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Café,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,Creperie,Dessert Shop,Dog Run,Farmers Market,Fish & Chips Shop,Fish Market,Flower Shop,French Restaurant,Furniture / Home Store,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Historic Site,History Museum,Hotel,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Liquor Store,Mediterranean Restaurant,Middle Eastern Restaurant,Movie Theater,Other Great Outdoors,Park,Pastry Shop,Pet Store,Restaurant,Rock Climbing Spot,Sandwich Place,Science Museum,Shopping Mall,Shopping Plaza,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Turkish Restaurant,Vietnamese Restaurant
0,Parkhoods,0.01,0.01,0.01,0.02,0.01,0.05,0.02,0.01,0.02,0.03,0.04,0.01,0.06,0.02,0.03,0.01,0.01,0.01,0.01,0.01,0.02,0.01,0.01,0.01,0.01,0.01,0.02,0.02,0.02,0.01,0.01,0.01,0.01,0.01,0.04,0.01,0.03,0.01,0.05,0.01,0.01,0.04,0.01,0.01,0.01,0.01,0.02,0.01,0.01,0.01,0.01,0.02,0.03,0.01,0.03,0.01,0.01,0.01


In [48]:
#Confirm the newsize
toronto_grouped.shape

(1, 59)

In [54]:
#Print each neighborhood along with the top 5 most common venues
num_top_venues=10
for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp=toronto_grouped[toronto_grouped['Neighborhood']==hood].T.reset_index()
    temp.columns=['venue', 'freq']
    temp=temp.iloc[1:]
    temp['freq']=temp['freq'].astype(float)
    temp=temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    
    print('\n')

----Parkhoods----
                       venue  freq
0       Caribbean Restaurant  0.06
1  Middle Eastern Restaurant  0.05
2                     Bakery  0.05
3        Japanese Restaurant  0.04
4                       Café  0.04
5                       Park  0.04
6                Supermarket  0.03
7                   Tea Room  0.03
8               Burger Joint  0.03
9               Liquor Store  0.03




Put The Above Into A Pandas Dataframe

In [50]:
#First, a function to sort the venues in descending order, before putting the above into a pandas dataframe
def return_most_common_venues(row, num_top_venues):
    row_categories=row.iloc[1:]
    row_categories_sorted=row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [55]:
#Create a new dataframe and display the top 10 venues for each neighborhood
num_top_venues=10
indicators=['st', 'nd', 'rd']

#Create columns according to number of top venues
columns=['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
#Create a new dataframe
toronto_venues_sorted=pd.DataFrame(columns=columns)
toronto_venues_sorted['Neighborhood']=toronto_grouped['Neighborhood']
for ind in np.arange(toronto_grouped.shape[0]):
    toronto_venues_sorted.iloc[ind,1:]=return_most_common_venues(toronto_grouped.iloc[ind,:], num_top_venues)
toronto_venues_sorted.head()    

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Parkhoods,Caribbean Restaurant,Middle Eastern Restaurant,Bakery,Japanese Restaurant,Park,Café,Burger Joint,Tea Room,Coffee Shop,Supermarket


Cluster The Neighborhoods

In [ ]:
#Run k-means to cluster the neighborhoods into 5 clusters
#Set number of clusters
kclusters=5
toronto_grouped_clustering=toronto_grouped.drop('Neighborhood', 1)

#Run k-means clustering
kmeans=KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

#Check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

Create a new DataFrame that includes the clusters as well as the top 10 venues for each Neighborhood

In [ ]:
#Add clustering labels
toronto_venues_sorted.insert(0, 'cluster Labels', kmeans.labels_)

toronto_merged=neighborhoods_data

#Merge toronto_grouped with neighborhoods_data to add latitude/longitude for each neighborhood
toronto_merged=toronto_merged.join(toronto_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() #check the last column!

Finally, Visualize the Resulting Cluster:

In [ ]:
#Create map
clustersmap=folium.Map(location=[latitude, longitude], zoom_start=11)

#Set color scheme for the clusters
x=np.arange(kclusters)
ys=[i + x +(i * x)**2 for i in range(kclusters)]
colors_array=cm.rainbow(np.linespace(0, 1, len(ys)))
rainbow=[colors.rgb2hex(i) for i in color_array]

#Add markers to the map
markers_colors=[]
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'],
                                 toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label=folium.Popup(str(poi) + Cluster + str(cluster), parse_html=True)
    folium.CircleMarker([lat, lon],
                       radius=5,
                       popup=label,
                       color=rainbow[cluster -1],
                       fill=True,
                       fill_color=rainbow[cluster -1],
                       fill_opacity=0.7).add_to(clustersmap)
    clustersmap